### References:
* https://flask-sqlalchemy.readthedocs.io/en/stable/
* https://docs.sqlalchemy.org/en/20/orm/inheritance.html#concrete-table-inheritance
* https://docs.sqlalchemy.org/en/20/_modules/examples/performance/bulk_inserts.html
* https://docs.sqlalchemy.org/en/20/orm/large_collections.html#bulk-insert-of-new-items
* https://docs.sqlalchemy.org/en/20/orm/extensions/asyncio.html

## Refactoring notes

- Dans le modèle SQLAlchmey PremiumFile, il manque la relation à une analyse (mais la table d'association existe bien). A ajouter :

```python
class PremiumFile(CommonMixin, Base):
    """Represents a historical premium file."""
    id: Mapped[int] = mapped_column(primary_key=True)
    client_id: Mapped[int] = mapped_column(ForeignKey("client.id"))
    client: Mapped["Client"] = relationship(back_populates="premiumfiles")

    # TODO: To be added
    analyses: Mapped[List[Analysis]] = relationship(
        secondary=lambda: analysis_premiumfile_table, back_populates="premiumfiles"
    )
```

- Dans le modèle SQLAlchemy FrequencySeverityModel, il manque la relation avec le premium file en input du modèle. A ajouter :

```python
class FrequencySeverityModel(ModelFile):
    id: Mapped[int] = mapped_column(ForeignKey("modelfile.id"), primary_key=True)
    threshold: Mapped[int] = mapped_column(nullable=False)
    lossfile_id: Mapped[int] = mapped_column(ForeignKey("histolossfile.id"))
    lossfile: Mapped["HistoLossFile"] = relationship()
    premiumfile_id: Mapped[int] = mapped_column(ForeignKey("premiumfile.id"))  # TODO: To be added
    premiumfile: Mapped["PremiumFile"] = relationship()  # TODO: To be added
```

- Dans la route qui crée un frequency-severity model file, il manque l'association du premium file au model file :

```python
def create_frequency_severity_model:
# Create the frequency-severity model
        modelfile = FrequencySeverityModel(
            model_type="frequency_severity_model",
            threshold=threshold,
            years_simulated=years_simulated,
            lossfile_id=lossfile_id,
            premiumfile_id=premiumfile_id,  # TODO: To be added
            frequencymodel=frequencymodel,
            severitymodel=severitymodel,
        )

```

- Dans les relationships premiumfiles, histolossfiles et modelfiles du modèle SQLAlchemy Analysis, il manque les back-populates vers analysis, ce qui crée une anomalie lors de la suppression d'un premiumfile, histolossfile ou modelfile :
```
Database error occurred while deleting HistoLossFile record: (psycopg2.errors.ForeignKeyViolation) ERREUR:  UPDATE ou DELETE sur la table « histolossfile » viole la contrainte de clé étrangère « analysis_histolossfile_histolossfile_id_fkey » de la table « analysis_histolossfile »
DETAIL:  La clé (id)=(1) est toujours référencée à partir de la table « analysis_histolossfile ».
```
- => A Ajouter :

```python
class Analysis(CommonMixin, Base):
    premiumfiles: Mapped[List["PremiumFile"]] = relationship(
        secondary=lambda: analysis_premiumfile_table, back_populates="analyses",    # TODO: Add back_populates
    )
    
    histolossfiles: Mapped[List["HistoLossFile"]] = relationship(
        secondary=lambda: analysis_histolossfile_table, back_populates="analyses"  # TODO: Add back_populates
    )
    modelfiles: Mapped[List["ModelFile"]] = relationship(
        secondary=lambda: analysis_modelfile_table, back_populates="analyses"  # TODO: Add back_populates
    )
```

- De même, dans les modèles SQLAlchemy PremiumFile, HistoLossFile et ModelFile, il manque les relationships vers analysis. A ajouter :

```python
class PremiumFile(CommonMixin, Base):
    analyses: Mapped[List[Analysis]] = relationship(
        secondary=lambda: analysis_premiumfile_table, back_populates="premiumfiles"
    )  # TODO: To be added
    
class HistoLossFile(CommonMixin, Base):
    analyses: Mapped[List[Analysis]] = relationship(
        secondary=lambda: analysis_histolossfile_table, back_populates="histolossfiles"
    )  # TODO: To be added

class ModelFile(CommonMixin, Base):
    analyses: Mapped[List[Analysis]] = relationship(
        secondary=lambda: analysis_modelfile_table, back_populates="modelfiles"
    )  # TODO: To be added
```

- The Pydantic classes FrequencyInput and SeverityInput need to be reviewed and refactored
- Start frequency and severity models parameters with index 0 :

```python
class FrequencyModel(CommonMixin, Base):
    parameter_0: Mapped[float] = mapped_column(nullable=False)  # TODO: To be added
    parameter_1: Mapped[float]
    parameter_2: Mapped[float]
    parameter_3: Mapped[float]
    parameter_4: Mapped[float]
    # parameter_5: Mapped[float]  # TODO: To be deleted

class SeverityModel(CommonMixin, Base):
    parameter_0: Mapped[float] = mapped_column(nullable=False)  # TODO: To be added
    parameter_1: Mapped[float]
    parameter_2: Mapped[float]
    parameter_3: Mapped[float]
    parameter_4: Mapped[float]
    # parameter_5: Mapped[float]  # TODO: To be deleted
```

- Utiliser les valeurs ModelType pour définir les identités polymorphiques liées au modèle SQLAlchemy ModelFile :

```python
class ModelType(Enum):  # TODO: To be added
    """Defines the supported loss models."""

    EMPIRICAL = "empirical"
    FREQUENCY_SEVERITY = "frequency_severity"  # TODO: Improve the value with underscores
    COMPOSITE_FREQUENCY_SEVERITY = "composite_frequency_severity"
    EXPOSURE_BASED = "exposure_based"


class ModelFile(CommonMixin, Base):


class EmpiricalModel(ModelFile):
    __mapper_args__ = {
        "polymorphic_identity": ModelType.EMPIRICAL,  # TODO: To be corrected
    }


class FrequencySeverityModel(ModelFile):
    __mapper_args__ = {
        "polymorphic_identity": ModelType.FREQUENCY_SEVERITY,  # TODO: To be corrected
    }
```

```python
from db.models import (
    Analysis,
    Client,
    FrequencyModel,
    FrequencySeverityModel,
    HistoLossFile,
    ModelType,  # TODO: To be added
    ModelYearLoss,
    PremiumFile,
    SeverityModel,
)

def create_frequency_severity_model
        # Create the frequency-severity model
        modelfile = FrequencySeverityModel(
            model_type=ModelType.FREQUENCY_SEVERITY.value,  # TODO: To be corrected
            threshold=threshold,
            years_simulated=years_simulated,
            lossfile_id=lossfile_id,
            premiumfile_id=premiumfile_id,
            frequencymodel=frequencymodel,
            severitymodel=severitymodel,
        )
```

- **Confusion entre input et output (cf. https://gitlab.com/ccr-re-df/products/app/backends/tarification-nonvie-backend/-/blob/dev/app/api/routes/model.py) :**

```python
async def generate_stochastic_year_loss_table_and_metadata(
    frequence_model_input: FrequencyModelOutput,
    severity_model_input: SeverityModelOutput,
    frequence_severity_model_input: FrequencySeverityModelInputExtend,
    threshold_input: float,
    analysis_id: int,
    user_id: int,
    lossfile_id: int,
    injected_model_service: ModelServiceDep,
) -> StochasticModelRouteResponse:
```

- Finally, create **specific** issues in Jira

## Test Engine

In [1]:
import time

from engine.model.frequency_severity import (
    DistributionInput,
    DistributionType,
    get_modelyearloss_frequency_severity,
)

In [2]:
threshold = 1000

frequency_input = DistributionInput(
    dist=DistributionType.POISSON,
    threshold=threshold,
    params=[3],
)

severity_input = DistributionInput(
    dist=DistributionType.PARETO,
    threshold=threshold,
    params=[2],
)

cat_share = 0.5
simulated_years = 100_000
modelfile_id = 1

start = time.perf_counter()

modelyearloss = get_modelyearloss_frequency_severity(
    frequency_input,
    severity_input,
    cat_share,
    simulated_years,
    modelfile_id,
)

print(f"Duration = {time.perf_counter() - start}")
print(f"Average Loss = {modelyearloss["loss"].mean()}")
print(f"Frequency = {len(modelyearloss["loss"]) / simulated_years}")
print(modelyearloss)

Duration = 0.1773916999809444
Average Loss = 1986.6839583027847
Frequency = 3.00068
shape: (300_068, 11)
┌───────┬─────┬──────┬───────────┬───┬────────────┬────────┬──────────────────┬──────────────┐
│ year  ┆ day ┆ loss ┆ loss_type ┆ … ┆ model_hash ┆ model  ┆ line_of_business ┆ modelfile_id │
│ ---   ┆ --- ┆ ---  ┆ ---       ┆   ┆ ---        ┆ ---    ┆ ---              ┆ ---          │
│ i64   ┆ i32 ┆ i64  ┆ str       ┆   ┆ object     ┆ object ┆ object           ┆ i64          │
╞═══════╪═════╪══════╪═══════════╪═══╪════════════╪════════╪══════════════════╪══════════════╡
│ 0     ┆ 152 ┆ 1534 ┆ cat       ┆ … ┆ null       ┆ null   ┆ null             ┆ 1            │
│ 0     ┆ 261 ┆ 1232 ┆ cat       ┆ … ┆ null       ┆ null   ┆ null             ┆ 1            │
│ 0     ┆ 91  ┆ 1699 ┆ cat       ┆ … ┆ null       ┆ null   ┆ null             ┆ 1            │
│ 0     ┆ 91  ┆ 1923 ┆ non_cat   ┆ … ┆ null       ┆ null   ┆ null             ┆ 1            │
│ 1     ┆ 61  ┆ 1000 ┆ cat       ┆ … ┆ n

## Test Backend

In [1]:
import time

from db.crud import (
    create_analysis,
    create_client,
    create_frequency_severity_model,
    create_historical_loss_file,
    create_premium_file,
    delete_db_record,
)
from db.models import (
    Analysis,
    Client,
    FrequencyModel,
    FrequencySeverityModel,
    HistoLossFile,
    ModelFile,
    ModelYearLoss,
    PremiumFile,
    SeverityModel,
    session,
)
from engine.model.frequency_severity import DistributionInput, DistributionType

In [2]:
# Create a client, an analysis, a premium file and a historical loss file
create_client(session, client_name="AXA")
create_analysis(session, client_id=1)
create_premium_file(session, analysis_id=1)
create_historical_loss_file(session, analysis_id=1)

Client 'AXA' with ID 1 added successfully.
Analysis with ID 1 added successfully.
Premium file with ID 1 added successfully.
Historical loss file with ID 1 added successfully.


In [3]:
# Create a frequency-severity model
start = time.perf_counter()
create_frequency_severity_model(
    session,
    analysis_id=1,
    lossfile_id=1,
    premiumfile_id=1,
    threshold=1000,
    frequency_input=DistributionInput(
        dist=DistributionType.POISSON,
        threshold=1000,
        params=[3, 0, 0, 0, 0],
    ),
    severity_input=DistributionInput(
        dist=DistributionType.PARETO,
        threshold=1000,
        params=[2, 0, 0, 0, 0],
    ),
    cat_share=0.5,
    years_simulated=10_000,
)

duration = time.perf_counter() - start
print(f"Total Duration = {duration}")

Time to create model file: 0.02 seconds
Time to flush the session: 0.00 seconds
Time to generate year loss data: 0.08 seconds
Time to insert year loss records into database: 1.28 seconds
Time to commit transaction: 0.01 seconds
Frequency-Severity Model with ID 1 created successfully.
Total Duration = 1.385781099786982


In [4]:
# DbModel = Analysis
# DbModel = PremiumFile
DbModel = HistoLossFile
# DbModel = FrequencyModel
# DbModel = SeverityModel
# DbModel = FrequencySeverityModel

delete_db_record(session, DbModel, 1)

HistoLossFile record with ID 1 has been deleted.


In [6]:
DbModel = Analysis
# DbModel = PremiumFile
# DbModel = HistoLossFile
# DbModel = FrequencyModel
# DbModel = SeverityModel
# DbModel = FrequencySeverityModel

delete_db_record(session, DbModel, 2)

Analysis record with ID 2 has been deleted.


- Problem with the deletion of frequency and severity models